In [1]:
import scipy.io as sio
import numpy as np
import os
import math
import tensorflow as tf
import keras
from keras.layers import Conv1D, Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization, Concatenate
from keras.callbacks import EarlyStopping
from keras.layers import Dropout, Input
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import matplotlib.pyplot as plt

C:\Users\user\anaconda3\envs\py\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\anaconda3\envs\py\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\anaconda3\envs\py\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\anaconda3\envs\py\lib\site-packages\tensorflow\python\framework\dtypes.

In [2]:
## GLOBAL VARIABLES
test_ratio = 0.25
windowSize = 5

In [3]:
def loadData():
    data_path = os.path.join(os.getcwd(),'data')
    data = sio.loadmat(os.path.join(data_path, 'Agucha_input.mat'))['x']
    labels = sio.loadmat(os.path.join(data_path, 'Agucha_output.mat'))['y']
    return data, labels

In [4]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [5]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [6]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = False):
    margin = int((windowSize - 1) / 2) # padding width = (filter_size-1)/2
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [7]:
def bandMapping (lamb, mu, sigma, strength, th, dim):
    y = []
    LL = 0
    UL = 0
    for j in range(0, dim):
#         print((strength * math.exp(-0.5 * math.pow(((lamb[j]-mu)/sigma),2))))
        if (strength * math.exp(-0.5 * math.pow(((lamb[j]-mu)/sigma),2))) <= th:
            LL =  j # LL is lower limit of the band
        elif (strength * math.exp(-0.5 * math.pow(((lamb[j]-mu)/sigma),2))) >= th:
            if j == dim-1: # this is evoked when the band lies at the right end of the spectrum
                UL = j
                break
            elif (strength * math.exp(-0.5 * math.pow(((lamb[j]-mu)/sigma),2)))  \
                > (strength * math.exp(-0.5 * math.pow(((lamb[j+1]-mu)/sigma),2))):
                UL =  j 
                break
    UL = UL + (UL - LL -1) # UL is upper limit of the band
    UL = min(UL, dim) # dim represents maximum number of channels
#     print(LL, UL)
    for k in range(LL, UL):
        y.append((strength * math.exp(-0.5 * math.pow(((lamb[k]-mu)/sigma),2))))
    return LL, UL, np.array(y)

In [8]:
X, y = loadData()
X = X[0:133, :] # to remove the pixels which had only 0 in the spectral dimension
y = y[0:133, :]
# Removing class 3 with 0 class (as there are not enough samples)
y[65, 28] = 0
y[65, 29] = 0
y[66, 27] = 0
y[66, 28] = 0
# Shifting all the classes with index 4 or more by 1 (as class 3 is no longer present)
for i in range(y.shape[0]): 
    for j in range(y.shape[1]):
        if y[i, j] >= 4:
            y[i, j] = y[i, j] - 1

In [9]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((17822, 5, 5, 148), (17822,))

In [11]:
data_path = os.path.join(os.getcwd(),'data')
t1 = sio.loadmat(os.path.join(data_path, 'matlab_matrix.mat'))['train']
t2 = sio.loadmat(os.path.join(data_path, 'matlab_matrix.mat'))['test']

In [13]:
t3 = []
t4 = []
t5 = []
t6 = []
for i in range(t1.shape[1]):
    if y[t1[0,i]*134 + t1[1,i]] > 0:
        t3.append(X[t1[0,i]*134 + t1[1,i]])
        t4.append(y[t1[0,i]*134 + t1[1,i]])
for i in range(t2.shape[1]):
    if y[t2[0,i]*134 + t2[1,i]] > 0:
        t5.append(X[t2[0,i]*134 + t2[1,i]])
        t6.append(y[t2[0,i]*134 + t2[1,i]])

In [14]:
t3 = np.array(t3)
t4 = np.array(t4)
t5 = np.array(t5)
t6 = np.array(t6)

In [15]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((13366, 5, 5, 148), (4456, 5, 5, 148), (13366,), (4456,))

In [16]:
Xtrain = t3
ytrain = t4
Xtest = t5
ytest = t6

In [17]:
np.unique(ytest)

array([1., 2., 3., 4., 5.])

In [18]:
ytrain = np_utils.to_categorical(ytrain)
ytrain.shape

(6791, 6)

In [19]:
bandPar = np.loadtxt("mu_sigma_strength.txt") # reading band parameters
Absp = bandPar.shape[0] # number of Absorption bands
lamb = np.loadtxt("wavelength_list.txt") # reading band parameters

In [20]:
mu = np.zeros(Absp)
sigma = np.zeros(Absp)
strength = np.zeros(Absp)
for i in range(0,Absp):
    mu[i] = bandPar[i,0]
    sigma[i] = bandPar[i,1]
    strength[i] = bandPar[i,2]
strength = 10000*strength 
Threshhold = 0.01

In [21]:
LL = np.zeros(Absp)
UL = np.zeros(Absp)
weight = []
for i in range(0, Absp): # Absp is number of absorption bands
    LL[i], UL[i], t1 = bandMapping (lamb, mu[i], sigma[i], strength[i], Threshhold, X.shape[3])
    weight.append(t1)
LL = LL.astype('int64')
UL = UL.astype('int64')

In [22]:
W = []
for i in range(0, Absp): # Absp is number of absorption bands
    for l in range(0, Xtrain.shape[0]): # Xtrain.shape[0] represents number of training patches
#         print("Band ", i+1, " patch", l)
        Y = []
        for j in range(0, windowSize): 
            for k in range(0, windowSize):
#                 print("Band ", i+1)
                t1 = 0
                t2 = []
                for m in range(LL[i], UL[i]):
                    t2.append(Xtrain[l,j,k,m] * weight[i][t1])
                    t1 = t1 + 1
                t2 = np.array(t2)
                Y.append(t2)
        Y = np.array(Y)
        Z = Y.reshape(Y.shape[0] * Y.shape[1],1)
        Z = Z.reshape(windowSize, windowSize, (UL[i]-LL[i]))
        if l == 0:
            V = np.zeros((Xtrain.shape[0], Z.shape[0], Z.shape[1], Z.shape[2]))
            V[l,:,:,:] = Z
        V[l,:,:,:] = Z
    W.append(V)

In [23]:
len(W)

45

In [24]:
for i in range(0,Absp):
    print(W[i].shape)

(6791, 5, 5, 11)
(6791, 5, 5, 7)
(6791, 5, 5, 5)
(6791, 5, 5, 3)
(6791, 5, 5, 17)
(6791, 5, 5, 43)
(6791, 5, 5, 48)
(6791, 5, 5, 42)
(6791, 5, 5, 33)
(6791, 5, 5, 9)
(6791, 5, 5, 11)
(6791, 5, 5, 81)
(6791, 5, 5, 99)
(6791, 5, 5, 137)
(6791, 5, 5, 3)
(6791, 5, 5, 75)
(6791, 5, 5, 42)
(6791, 5, 5, 42)
(6791, 5, 5, 8)
(6791, 5, 5, 13)
(6791, 5, 5, 3)
(6791, 5, 5, 79)
(6791, 5, 5, 59)
(6791, 5, 5, 71)
(6791, 5, 5, 42)
(6791, 5, 5, 32)
(6791, 5, 5, 23)
(6791, 5, 5, 39)
(6791, 5, 5, 21)
(6791, 5, 5, 89)
(6791, 5, 5, 108)
(6791, 5, 5, 42)
(6791, 5, 5, 83)
(6791, 5, 5, 13)
(6791, 5, 5, 49)
(6791, 5, 5, 63)
(6791, 5, 5, 123)
(6791, 5, 5, 5)
(6791, 5, 5, 129)
(6791, 5, 5, 23)
(6791, 5, 5, 51)
(6791, 5, 5, 131)
(6791, 5, 5, 96)
(6791, 5, 5, 65)
(6791, 5, 5, 42)


In [25]:
output_units = 5

inp = [0]*Absp
conv1 = [0]*Absp
conv2 = [0]*Absp
flt = [0]*Absp

for i in range (0, Absp):
    inp[i] = Input(shape = (W[i].shape[1], W[i].shape[2], W[i].shape[3], 1))
    conv1[i] = Conv3D(filters=20, kernel_size=(3, 3, 3), activation='relu')(inp[i])
#     conv2[i] = Conv3D(filters=20, kernel_size=(2, 2, 2), activation='relu')(conv1[i])
    conv3d_shape = conv1[i]._keras_shape
    print(conv1[i]._keras_shape)
    conv1[i] = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv1[i])
    conv2[i] = Conv2D(64, (3, 3), activation='relu')(conv1[i])
#     conv4[i] = Conv2D(64, (2, 2), activation='relu')(conv3[i])
    flt[i] = Flatten()(conv2[i])

mrg = Concatenate()(flt)

dense1 = Dense(256, activation='relu')(mrg)
# dense2 = Dense(units=128, activation='relu')(dense1)

op = Dense(6, activation='softmax')(dense1)

model = Model(input=inp, output=op)

adam = Adam(lr=0.001, decay=1e-06)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

(None, 3, 3, 9, 20)
(None, 3, 3, 5, 20)
(None, 3, 3, 3, 20)
(None, 3, 3, 1, 20)
(None, 3, 3, 15, 20)
(None, 3, 3, 41, 20)
(None, 3, 3, 46, 20)
(None, 3, 3, 40, 20)
(None, 3, 3, 31, 20)
(None, 3, 3, 7, 20)
(None, 3, 3, 9, 20)
(None, 3, 3, 79, 20)
(None, 3, 3, 97, 20)
(None, 3, 3, 135, 20)
(None, 3, 3, 1, 20)
(None, 3, 3, 73, 20)
(None, 3, 3, 40, 20)
(None, 3, 3, 40, 20)
(None, 3, 3, 6, 20)
(None, 3, 3, 11, 20)
(None, 3, 3, 1, 20)
(None, 3, 3, 77, 20)
(None, 3, 3, 57, 20)
(None, 3, 3, 69, 20)
(None, 3, 3, 40, 20)
(None, 3, 3, 30, 20)
(None, 3, 3, 21, 20)
(None, 3, 3, 37, 20)
(None, 3, 3, 19, 20)
(None, 3, 3, 87, 20)
(None, 3, 3, 106, 20)
(None, 3, 3, 40, 20)
(None, 3, 3, 81, 20)
(None, 3, 3, 11, 20)
(None, 3, 3, 47, 20)
(None, 3, 3, 61, 20)
(None, 3, 3, 121, 20)
(None, 3, 3, 3, 20)
(None, 3, 3, 127, 20)
(None, 3, 3, 21, 20)
(None, 3, 3, 49, 20)
(None, 3, 3, 129, 20)
(None, 3, 3, 94, 20)
(None, 3, 3, 63, 20)
(None, 3, 3, 40, 20)
Model: "model_1"
___________________________________________

C:\Users\user\anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [26]:
xtrain_list = []
for i in range(0, Absp):
    xtrain_list.append(W[i])
len(xtrain_list)
for i in range(0, Absp):
    xtrain_list[i] = xtrain_list[i].reshape(-1, windowSize, windowSize, xtrain_list[i].shape[3], 1)

In [27]:
W = []
for i in range(0, Absp): # Absp is number of absorption bands
    for l in range(0, Xtest.shape[0]): # Xtrain.shape[0] represents number of training patches
#         print("Band ", i+1, " patch", l)
        Y = []
        for j in range(0, windowSize): 
            for k in range(0, windowSize):
#                 print("Band ", i+1)
                t1 = 0
                t2 = []
                for m in range(LL[i], UL[i]):
                    t2.append(Xtest[l,j,k,m] * weight[i][t1])
                    t1 = t1 + 1
                t2 = np.array(t2)
                Y.append(t2)
        Y = np.array(Y)
        Z = Y.reshape(Y.shape[0] * Y.shape[1],1)
        Z = Z.reshape(windowSize, windowSize, (UL[i]-LL[i]))
        if l == 0:
            V = np.zeros((Xtest.shape[0], Z.shape[0], Z.shape[1], Z.shape[2]))
            V[l,:,:,:] = Z
        V[l,:,:,:] = Z
    W.append(V)

In [28]:
xtest_list = []
for i in range(0, Absp):
    xtest_list.append(W[i])
for i in range(0, Absp):
    xtest_list[i] = xtest_list[i].reshape(-1, windowSize, windowSize, xtest_list[i].shape[3], 1)

In [29]:
ytest = np_utils.to_categorical(ytest)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
history = model.fit(xtrain_list, ytrain, validation_data=(xtest_list, ytest), batch_size = 64, epochs=200, 
                    verbose=1, callbacks=[es, mc])


Train on 6791 samples, validate on 1201 samples
Epoch 1/200
6791/6791 [==============================] - 279s 41ms/step - loss: 406460.3750 - accuracy: 0.4365 - val_loss: 10760.2571 - val_accuracy: 0.2614

Epoch 00001: val_accuracy improved from -inf to 0.26145, saving model to best_model.h5
Epoch 2/200
6791/6791 [==============================] - 266s 39ms/step - loss: 5625.1296 - accuracy: 0.5101 - val_loss: 10708.1151 - val_accuracy: 0.3963

Epoch 00002: val_accuracy improved from 0.26145 to 0.39634, saving model to best_model.h5
Epoch 3/200
6791/6791 [==============================] - 267s 39ms/step - loss: 3789.9628 - accuracy: 0.5772 - val_loss: 3081.3488 - val_accuracy: 0.5071

Epoch 00003: val_accuracy improved from 0.39634 to 0.50708, saving model to best_model.h5
Epoch 4/200
6791/6791 [==============================] - 269s 40ms/step - loss: 1695.6707 - accuracy: 0.6413 - val_loss: 2504.2964 - val_accuracy: 0.6137

Epoch 00004: val_accuracy improved from 0.50708 to 0.61366, 

6791/6791 [==============================] - 262s 39ms/step - loss: 75.2113 - accuracy: 0.9639 - val_loss: 1520.6358 - val_accuracy: 0.5953

Epoch 00038: val_accuracy did not improve from 0.63031
Epoch 39/200
6791/6791 [==============================] - 260s 38ms/step - loss: 41.1245 - accuracy: 0.9661 - val_loss: 1376.0049 - val_accuracy: 0.6236

Epoch 00039: val_accuracy did not improve from 0.63031
Epoch 40/200
6791/6791 [==============================] - 264s 39ms/step - loss: 22.0205 - accuracy: 0.9747 - val_loss: 1512.6115 - val_accuracy: 0.6028

Epoch 00040: val_accuracy did not improve from 0.63031
Epoch 41/200
6791/6791 [==============================] - 264s 39ms/step - loss: 22.7114 - accuracy: 0.9773 - val_loss: 1487.1969 - val_accuracy: 0.6162

Epoch 00041: val_accuracy did not improve from 0.63031
Epoch 42/200
6791/6791 [==============================] - 264s 39ms/step - loss: 19.3181 - accuracy: 0.9828 - val_loss: 1496.2291 - val_accuracy: 0.6178

Epoch 00042: val_accura

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(xtrain_list, ytrain, verbose=0)
_, test_acc = model.evaluate(xtest_list, ytest, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

Commented

In [ ]:
# model.save('model6.h5')

In [ ]:
from keras.models import load_model
model = load_model('best_model.h5')

In [ ]:
# Xtest.shape

In [ ]:
ytest.shape

In [ ]:
# for i in range(0, Absp):
#     xtest_list[i] = xtest_list[i].reshape(-1, windowSize, windowSize, xtest_list[i].shape[3], 1)

In [ ]:
xtest_list[1].shape

In [ ]:
# W = []
# for i in range(0, Absp): # Absp is number of absorption bands
#     for l in range(0, Xtest.shape[0]): # Xtrain.shape[0] represents number of training patches
# #         print("Band ", i+1, " patch", l)
#         Y = []
#         for j in range(0, windowSize): 
#             for k in range(0, windowSize):
# #                 print("Band ", i+1)
#                 t1 = 0
#                 t2 = []
#                 for m in range(LL[i], UL[i]):
#                     t2.append(Xtest[l,j,k,m] * weight[i][t1])
#                     t1 = t1 + 1
#                 t2 = np.array(t2)
#                 Y.append(t2)
#         Y = np.array(Y)
#         Z = Y.reshape(Y.shape[0] * Y.shape[1],1)
#         Z = Z.reshape(windowSize, windowSize, (UL[i]-LL[i]))
#         if l == 0:
#             V = np.zeros((Xtest.shape[0], Z.shape[0], Z.shape[1], Z.shape[2]))
#             V[l,:,:,:] = Z
#         V[l,:,:,:] = Z
#     W.append(V)

In [ ]:
# for i in range(0,Absp):
#     print(W[i].shape)

In [ ]:
# K = X.shape[3]
# Xtest = Xtest.reshape(-1, windowSize, windowSize, K, 1)
# # Xtest = Xtest.reshape(-1, windowSize, windowSize, K)
# Xtest.shape

In [ ]:
# ytest = np_utils.to_categorical(ytest, num_classes=6)
# ytest.shape

In [ ]:
# Xtest.shape

In [ ]:
# xtest_list = []
# for i in range(0, Absp):
#     xtest_list.append(W[i])
# len(xtest_list)

In [ ]:
Y_pred_test = model.predict(xtest_list)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [ ]:
ytest.shape

In [ ]:
# from operator import truediv

In [ ]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
def reports (X_test,y_test):
    #start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    target_names = ['qms', 'pegmatite', 'gbsg', 'garnet', 'galena']
    
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
from operator import truediv
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(xtest_list,ytest)

In [ ]:
# classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(xtest_list,ytest)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report.txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

In [ ]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    return patch

In [ ]:
# load the original image
X, y = loadData()
X = X[0:133, :] # to remove the pixels which had only 0 in the spectral dimension
y = y[0:133, :]
# Removing class 3 with 0 class (as there are not enough samples)
y[65, 28] = 0
y[65, 29] = 0
y[66, 27] = 0
y[66, 28] = 0
# Shifting all the classes with index 4 or more by 1 (as class 3 is no longer present)
for i in range(y.shape[0]): 
    for j in range(y.shape[1]):
        if y[i, j] >= 4:
            y[i, j] = y[i, j] - 1

In [ ]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
# numComponents = K

In [ ]:
X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
outputs = np.zeros((height,width))
for ii in range(height):
    for jj in range(width):
        target = int(y[ii,jj])
#         print("target = ", target, "i = ", i, "j = ", j)
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,ii,jj)
            W = []
            for i in range(0, Absp): # Absp is number of absorption bands
                Y = []
                for j in range(0, windowSize): 
                    for k in range(0, windowSize):
                        t1 = 0
                        t2 = []
                        for m in range(LL[i], UL[i]):
                            t2.append(image_patch[j,k,m] * weight[i][t1])
                            t1 = t1 + 1
                        t2 = np.array(t2)
                        Y.append(t2)
                Y = np.array(Y)
                Z = Y.reshape(Y.shape[0] * Y.shape[1],1)
                Z = Z.reshape(windowSize, windowSize, (UL[i]-LL[i]))
                W.append(Z)
#             print(len(W))
            X_test_image = []
            for i in range(0, Absp):
                X_test_image.append(W[i])
            for i in range(0, Absp):
                X_test_image[i] =  X_test_image[i].reshape(-1, windowSize, windowSize, X_test_image[i].shape[2], 1).astype('float32') 
#             X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')                                   
#             print("i = ", i, "j = ", j)
            prediction = (model.predict(X_test_image))
#             print("prediction1 = ", prediction)
            prediction = np.argmax(prediction, axis=1)
#             print("prediction = ", prediction)
            outputs[ii][jj] = prediction
#             print("outputs = ", outputs[ii][jj], i, j)

In [ ]:
for i in range(0,Absp):
    print(X_test_image[i].shape)

In [ ]:
X_test_image[0]

In [ ]:
import spectral

In [ ]:
ground_truth = spectral.imshow(classes = y,figsize =(7,7))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [ ]:
spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors)

In [ ]:
labels = np.array(y).reshape((133,134))
labelDictionary={0:'Tailing pond', 1:'qms', 2:'pegmatite', 3:'gbsg', 4:'garnet', 5:'galena'}

In [ ]:
from matplotlib import patches
from spectral import spy_colors

In [ ]:
labelPatches = [ patches.Patch(color=spy_colors[x]/255.,
                 label=labelDictionary[x]) for x in np.unique(labels) ]

In [ ]:
plt.legend(handles=labelPatches, ncol=2, fontsize='medium', 
           loc='upper center', bbox_to_anchor=(0.5, -0.05));

In [ ]:
model.summary()

In [ ]:
file_name = "result.txt"

with open(file_name, 'w') as x_file:
    x_file.write(model.summary())

In [ ]:
outputs.shape

In [ ]:
y

In [ ]:
ytest

In [ ]:
y_pred_test

In [ ]:
y_pred_test

In [ ]:
r1 = np.argmax(ytest, axis=1)

In [ ]:
r1

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(r1, y_pred_test)
class_names = ['qms', 'pegmatite', 'gbsg', 'garnet', 'galena']


# Visualizing the results
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Plot normalized confusion matrix
plt.figure(figsize=(7, 7))
plot_confusion_matrix(cm, classes=class_names, normalize=False, title='Confusion matrix')
plt.show()

In [ ]:
score1 = model.evaluate(xtrain_list, ytrain, batch_size=32)

In [ ]:
Train_Loss =  score1[0]*100
Train_accuracy = score1[1]*100

In [ ]:
Train_accuracy

In [ ]:
spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors, dpi = 300)

In [ ]:
fig = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))
fig.savefig("predictions.jpg", format="jpg", dpi=1200)

In [ ]:
Y_pred_train = model.predict(xtrain_list)
y_pred_train = np.argmax(Y_pred_train, axis=1)

In [ ]:
r1 = np.argmax(ytrain, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(r1, y_pred_train)
class_names = ['qms', 'pegmatite', 'gbsg', 'garnet', 'galena']


# Visualizing the results
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


# Plot normalized confusion matrix
plt.figure(figsize=(7, 7))
plot_confusion_matrix(cm, classes=class_names, normalize=False, title='Confusion matrix')
plt.show()